In [1]:
import json
import urllib.request
import urllib.parse
import csv
import time
import numpy as np

In [2]:
list_done=[]

In [3]:
#10000/16=625 (maximum pages of id list)
start_t=0
loop_t=625

#address for downloading id list
url_0='https://api.scratch.mit.edu/explore/projects?limit=16&offset='
url_1='0'
url_2='&language=en&mode=recent&q=*'


#address for downloading json file
url_a='https://cdn.projects.scratch.mit.edu/internalapi/project/'
url_b='0'
url_c='/get/'

#http://projects.scratch.mit.edu/internalapi/project/PROJECT-ID/get/

url_d='http://projects.scratch.mit.edu/internalapi/project/'

#https://projects.scratch.mit.edu/340297329/get

url_e='https://projects.scratch.mit.edu/'



In [4]:
#some random user agents
user_agent0 = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
user_agent1 = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:70.0) Gecko/20100101 Firefox/70.0'
user_agent2 = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
user_agent3 = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.137 Safari/537.36 LBBROWSER'
user_agent4 = 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0'
user_agent5 = 'Opera/9.25 (Windows NT 5.1; U; en)'
user_agent6 = 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12'
user_agent7 = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'
user_agent=[user_agent0,user_agent1,user_agent2,user_agent3,user_agent4,user_agent5,user_agent6,user_agent7]

In [5]:
def CheckIdAddress(j,url_0,url_2,lag,num_retries=1):
    url_1=str(0+16*j)
    url_f=url_0+url_1+url_2
    #if num_retries==1:
        #print('Page:', j+1,', Offset=',16*j)
    rnd = np.random.randint(0,8,size=1)
    headers = { 'User-Agent' : user_agent[int(rnd)] }
    try: 
        if lag > 8:
            time.sleep(1)
            lag=0
        req=urllib.request.Request(url_f, headers=headers)
        html=urllib.request.urlopen(url_f)
        lag+=1
    except:
        print('Retry number:',num_retries)
        if num_retries > 9:
            time.sleep(1)
            num_retries=1
        return CheckIdAddress(j,url_0,url_2,lag,num_retries+1)

    else:
        hjson = json.loads(html.read())
        return hjson

In [6]:
def GetRecentID(start_t,loop_t,user_agent,url_0,url_1,url_2):
    print('Start scraping the list of recent 10000 project IDs.',time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    list_recent_id=[]
    lag=0
    for j in range(start_t,loop_t):
        #if lag > 9:
        #    time.sleep(1)
        #    lag=0
        hjson=CheckIdAddress(j,url_0,url_2,lag)
        
        for i in range(0,16):
            list_recent_id.append(hjson [i]['id'])
    print('ID scraping complete.',time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    return list_recent_id  

In [7]:
def CheckURL(json_url,user_agent):
    rnd = np.random.randint(0,8,size=1)
    headers = { 'User-Agent' : user_agent[int(rnd)] }
    try:
        req=urllib.request.Request(json_url, headers=headers)
        response = urllib.request.urlopen(json_url)
    except: 
        dum='---'
        return dum
    else:
        req = response.read()
        return req 

In [8]:
def DownloadJson(list_recent_id,url_a,url_c,url_d,url_e,user_agent,list_done,i,num_retries=1):
    #len_list=len(list_recent_id)
    #print(len_list)
    #for i in range(0,10000):
    #print(i)
    url_b=str(list_recent_id[i])
    json_url=url_a+url_b+url_c
    req=CheckURL(json_url,user_agent)
    if req=='---':
        #print(url_b,'does not exist.')
        #a=0
        json_url=url_d+url_b+url_c
        req_1=CheckURL(json_url,user_agent)
        if req_1=='---':
            json_url=url_e+url_b+url_c
            req_2=CheckURL(json_url,user_agent)
            if req_2=='---':
                if num_retries < 10:
                    num_retries+=1
                    
                    time.sleep(1)
                        
                    return DownloadJson(list_recent_id,url_a,url_c,url_d,url_e,user_agent,list_done,i,num_retries)
                else:
                    print(url_b,'does not exist.')
            else:
                if url_b not in list_done:
                    with open('projects_json/'+url_b+'.json', 'wb') as f:
                        f.write(req_2)
                list_done.append(url_b)    
        else:
            if url_b not in list_done:
                with open('projects_json/'+url_b+'.json', 'wb') as f:
                    f.write(req_1)
                list_done.append(url_b)
    else:
        if url_b not in list_done:
            with open('projects_json/'+url_b+'.json', 'wb') as f:
                f.write(req)
            list_done.append(url_b)
            #print('Download:',url_b)
        else:
            #print(url_b,' is already downloaded.')
            a=0
            
    #print('Json file download complete.', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    return list_done

In [ ]:
#main
vol_max=500000
vol_t=420000

lag=0
num_c=0
num_t=0

while len(list_done) < vol_t:
    
    #download id
    list_recent_id=GetRecentID(start_t,loop_t,user_agent,url_0,url_1,url_2)
    #print(len(list_recent_id))
    
    #download json files
    for i in range(0,len(list_recent_id)):
        lag+=1
        if lag >8:
            time.sleep(1)
            lag=0
        
        num_c+=1
        num_t+=1
        if num_t==1000:  
            print(num_c)
            num_t=0
            
        list_done=DownloadJson(list_recent_id,url_e,url_c,url_d,url_a,user_agent,list_done,i)
    
    print('Json file download complete.', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    num_c=0
    num_t=0
    print('Sleep start.', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    time.sleep(3600)
    print('Sleep finished.', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

Start scraping the list of recent 10000 project IDs. 2019-12-22 04:28:49
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 2
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 2
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 1
Retry number: 1
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 1
Retry number: 2
Retry number: 1
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8

355756881 does not exist.
355625460 does not exist.
5000
355753979 does not exist.
355753398 does not exist.
6000
7000
355734956 does not exist.
355732863 does not exist.
8000
9000
10000
Json file download complete. 2019-12-22 21:51:58
Sleep start. 2019-12-22 21:51:58
Sleep finished. 2019-12-22 22:51:58
Start scraping the list of recent 10000 project IDs. 2019-12-22 22:51:58
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry numb

Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 1
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Ret

Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Ret

Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 1
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Retry number: 3
Retry number: 4
Retry number: 5
Retry number: 6
Retry number: 7
Retry number: 8
Retry number: 9
Retry number: 10
Retry number: 2
Re

In [18]:
type(len(list_recent_id))

int

In [22]:
for i in range(0,10000):
        list_done=DownloadJson(list_recent_id,url_e,url_c,url_d,url_a,user_agent,list_done,i)
    
print('Json file download complete.', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

341476046 does not exist.
341475964 does not exist.
341436575 does not exist.
341476613 does not exist.
337737579 does not exist.
341476508 does not exist.
341475077 does not exist.
341476939 does not exist.
341474623 does not exist.
341475132 does not exist.
341476528 does not exist.
341476480 does not exist.
340945682 does not exist.
341349136 does not exist.
341462121 does not exist.
341461335 does not exist.
341475971 does not exist.
341475833 does not exist.
341476200 does not exist.
341475402 does not exist.
341475578 does not exist.
340454161 does not exist.
341383511 does not exist.
341476107 does not exist.
341474181 does not exist.
339949712 does not exist.
341477921 does not exist.
341473111 does not exist.
313981860 does not exist.
341470507 does not exist.
341475919 does not exist.
337746353 does not exist.
341476965 does not exist.
341475428 does not exist.
322465978 does not exist.
341349136 does not exist.
341471778 does not exist.
341472567 does not exist.
341474018 do

341461821 does not exist.
340550407 does not exist.
341162684 does not exist.
341464723 does not exist.
341464168 does not exist.
341464732 does not exist.
341460853 does not exist.
341464444 does not exist.
341464184 does not exist.
341464065 does not exist.
341462688 does not exist.
341459471 does not exist.
336675793 does not exist.
341462049 does not exist.
328392737 does not exist.
341464309 does not exist.
341462436 does not exist.
341463776 does not exist.
341462724 does not exist.
324187560 does not exist.
341058927 does not exist.
341460506 does not exist.
341456131 does not exist.
331028780 does not exist.
341465076 does not exist.
337679124 does not exist.
341460882 does not exist.
341461056 does not exist.
341461636 does not exist.
341457295 does not exist.
341463094 does not exist.
341454027 does not exist.
341447011 does not exist.
341058927 does not exist.
341461067 does not exist.
341454027 does not exist.
341457955 does not exist.
341463356 does not exist.
337679124 do

331163933 does not exist.
341342338 does not exist.
341453491 does not exist.
341337692 does not exist.
341452382 does not exist.
341454703 does not exist.
341447388 does not exist.
341373388 does not exist.
341186927 does not exist.
339194597 does not exist.
341455481 does not exist.
341449360 does not exist.
334612770 does not exist.
341450907 does not exist.
341452801 does not exist.
341451411 does not exist.
338146159 does not exist.
341452723 does not exist.
341452705 does not exist.
339986481 does not exist.
341453209 does not exist.
341447162 does not exist.
341455194 does not exist.
341455348 does not exist.
341454721 does not exist.
341452134 does not exist.
341447243 does not exist.
341452055 does not exist.
341445227 does not exist.
341452909 does not exist.
341454710 does not exist.
341452862 does not exist.
341452236 does not exist.
341443654 does not exist.
340981118 does not exist.
341410343 does not exist.
341455364 does not exist.
341448716 does not exist.
341450967 do

341426804 does not exist.
341449705 does not exist.
341442957 does not exist.
341449262 does not exist.
341450727 does not exist.
341450239 does not exist.
339327894 does not exist.
341448828 does not exist.
341452596 does not exist.
341449219 does not exist.
341450671 does not exist.
341450641 does not exist.
341448335 does not exist.
341410809 does not exist.
341448307 does not exist.
341432685 does not exist.
341450351 does not exist.
341448248 does not exist.
341450443 does not exist.
341329194 does not exist.
341449025 does not exist.
341447591 does not exist.
341450043 does not exist.
341450923 does not exist.
340284935 does not exist.
341450505 does not exist.
341450922 does not exist.
341446593 does not exist.
341449315 does not exist.
341450899 does not exist.
341450302 does not exist.
341447446 does not exist.
341448116 does not exist.
341447483 does not exist.
341450808 does not exist.
341448095 does not exist.
341320720 does not exist.
340069403 does not exist.
341448402 do

341427674 does not exist.
334328803 does not exist.
341440000 does not exist.
341444667 does not exist.
341444161 does not exist.
341422173 does not exist.
330974674 does not exist.
341088463 does not exist.
341444332 does not exist.
341417758 does not exist.
341441351 does not exist.
341435715 does not exist.
341434764 does not exist.
341442803 does not exist.
341441399 does not exist.
341443210 does not exist.
341435576 does not exist.
341445894 does not exist.
341442028 does not exist.
341442083 does not exist.
341441400 does not exist.
341446002 does not exist.
341438853 does not exist.
341427308 does not exist.
341446329 does not exist.
341435336 does not exist.
341441863 does not exist.
341442684 does not exist.
341441881 does not exist.
341441853 does not exist.
341445485 does not exist.
335755287 does not exist.
341444776 does not exist.
341439936 does not exist.
341444864 does not exist.
341445197 does not exist.
341444841 does not exist.
341430331 does not exist.
341444571 do

...

341307193 does not exist.
341402555 does not exist.
341332135 does not exist.
341121912 does not exist.
341404593 does not exist.
341400924 does not exist.
341394514 does not exist.
280066070 does not exist.
341400301 does not exist.
328451560 does not exist.
341402435 does not exist.
341413070 does not exist.
340422363 does not exist.
341405604 does not exist.
341407676 does not exist.
341403949 does not exist.
341405766 does not exist.
341403829 does not exist.
341401186 does not exist.
341400948 does not exist.
341412688 does not exist.
341413032 does not exist.
341410206 does not exist.
341405551 does not exist.
341320021 does not exist.
341407616 does not exist.
341409552 does not exist.
341405399 does not exist.
341412733 does not exist.
341408157 does not exist.
335016975 does not exist.
341404208 does not exist.
334174813 does not exist.
341407071 does not exist.
341166512 does not exist.
339192636 does not exist.
337678117 does not exist.
341402727 does not exist.
341404162 do

341404400 does not exist.
341401510 does not exist.
341409226 does not exist.
339600931 does not exist.
341402095 does not exist.
341405863 does not exist.
334113239 does not exist.
341398581 does not exist.
341405621 does not exist.
338758284 does not exist.
341401948 does not exist.
341407621 does not exist.
341380244 does not exist.
341402227 does not exist.
341399170 does not exist.
341059718 does not exist.
341404089 does not exist.
341026832 does not exist.
341403114 does not exist.
341295563 does not exist.
341402123 does not exist.
341407888 does not exist.
341409047 does not exist.
341301248 does not exist.
341403239 does not exist.
341297268 does not exist.
341403129 does not exist.
341397415 does not exist.
341402994 does not exist.
341400324 does not exist.
341383728 does not exist.
341394977 does not exist.
341405320 does not exist.
341405855 does not exist.
341307732 does not exist.
339590950 does not exist.
341401692 does not exist.
341406194 does not exist.
341403875 do

341399820 does not exist.
341399639 does not exist.
341399920 does not exist.
341400140 does not exist.
213189069 does not exist.
341401538 does not exist.
341398820 does not exist.
341398184 does not exist.
339584756 does not exist.
340497555 does not exist.
323704459 does not exist.
341404477 does not exist.
341402137 does not exist.
341394051 does not exist.
341405772 does not exist.
341402989 does not exist.
225299121 does not exist.
316836094 does not exist.
341402852 does not exist.
341396905 does not exist.
341329653 does not exist.
341402865 does not exist.
340753895 does not exist.
341394543 does not exist.
341402431 does not exist.
341402535 does not exist.
341400032 does not exist.
341383752 does not exist.
341285807 does not exist.
341397555 does not exist.
341398404 does not exist.
315204049 does not exist.
341369595 does not exist.
341399241 does not exist.
341113233 does not exist.
341394543 does not exist.
336934268 does not exist.
341397153 does not exist.
341397729 do

341396378 does not exist.
340739832 does not exist.
341386208 does not exist.
340392618 does not exist.
341399318 does not exist.
341396321 does not exist.
341188181 does not exist.
341397332 does not exist.
341391152 does not exist.
341396576 does not exist.
338807707 does not exist.
341399019 does not exist.
341399174 does not exist.
341394047 does not exist.
341398289 does not exist.
341378815 does not exist.
341398217 does not exist.
341399123 does not exist.
341399019 does not exist.
341399760 does not exist.
341387558 does not exist.
341378815 does not exist.
341392208 does not exist.
341399714 does not exist.
341399123 does not exist.
341378426 does not exist.
341344757 does not exist.
329320830 does not exist.
341322571 does not exist.
341397657 does not exist.
341390380 does not exist.
327554016 does not exist.
341399075 does not exist.
341397725 does not exist.
341397725 does not exist.
341396102 does not exist.
341289579 does not exist.
341386883 does not exist.
339519276 do

341392116 does not exist.
341389670 does not exist.
339592604 does not exist.
337865555 does not exist.
341394434 does not exist.
341391410 does not exist.
341395243 does not exist.
341383638 does not exist.
341389454 does not exist.
341394257 does not exist.
341384862 does not exist.
341395506 does not exist.
341395716 does not exist.
341391220 does not exist.
341393027 does not exist.
341373640 does not exist.
341385759 does not exist.
341390396 does not exist.
341387165 does not exist.
341392216 does not exist.
339592604 does not exist.
341387738 does not exist.
341391645 does not exist.
341391496 does not exist.
341395243 does not exist.
341387151 does not exist.
302562656 does not exist.
341341035 does not exist.
341392998 does not exist.
341391071 does not exist.
341232238 does not exist.
341395518 does not exist.
341041861 does not exist.
341391377 does not exist.
341388611 does not exist.
331638171 does not exist.
340308999 does not exist.
339626544 does not exist.
341384516 do

In [ ]:
#Start scraping the list of recent 10000 project IDs. 2019-11-02 23:10:35